In [ ]:
import os
import sys
import json
import datetime

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
data_path = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_loads.csv'))
df_raw_data = pd.read_csv(data_path, parse_dates=True, index_col='Date/Time')

In [ ]:
df_sim_input_data = df_raw_data[['Inst. HT [1] [W]', 'mdot [kg/s]', 'Outlet 1 [C]', 'Inlet 1 [C]']].copy(deep=True)
df_sim_input_data.rename(columns = {'Inst. HT [1] [W]': 'Exp. HT Rate [W]', 
                                    'Outlet 1 [C]': 'Exp. Inlet [C]', 
                                    'Inlet 1 [C]': 'Exp. Outlet [C]'}, inplace=True)
df_sim_input_data.to_csv(join(cwd, 'input_data_high.csv'))

In [ ]:
df_sim_input_data['time'] = pd.to_timedelta(df_sim_input_data.index)
df_sim_input_data['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_sim_input_data['time'] - df_sim_input_data['time'][0])
df_sim_input_data.set_index('time', inplace=True)
df_sim_input_data.index.rename('Date/Time', inplace=True)

In [ ]:
d = glhe.utilities.functions.load_json('MFRTRT_STS_input.json')
d['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, 'g.csv'))
d['temperature-profile'][0]['path'] = norm(join(cwd, 'input_data_high.csv'))
d['flow-profile'][0]['path'] = norm(join(cwd, 'input_data_high.csv'))
d['simulation']['output-path'] = cwd
d['simulation']['output-csv-name'] = 'out_high.csv'

d['simulation']['time-step'] = 15
d['simulation']['runtime'] = 3600 * 50
d['borehole-definitions'][0]['number-iterations'] = 2
d['borehole-definitions'][0]['fraction-grout'] = 0.25
d['borehole-definitions'][0]['segments'] = 1

In [ ]:
with open(join(cwd, 'in_high.json'), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in_high.json').simulate()

In [ ]:
df = pd.read_csv('out_high.csv', index_col=0, parse_dates=True)

In [ ]:
start_time = '2019-01-01 00:00:00'
end_time = '2019-01-01 02:00:00'

ax = df_sim_input_data[['Exp. Inlet [C]', 'Exp. Outlet [C]']].loc[start_time:end_time].plot(style='x--')
df[['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]',
    'GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]',
    'GroundHeatExchangerSTS:GHE 1:Borehole Wall Temp. [C]']].loc[start_time:end_time].plot(ax=ax)

In [ ]:
df['Outlet Temp Error [C]'] = df['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_sim_input_data['Exp. Outlet [C]']

In [ ]:
'RMSE: {:0.4f} [C]'.format(np.mean(df['Outlet Temp Error [C]'] ** 2) ** 0.5)

In [ ]:
'MBE: {:0.4f} [C]'.format(np.mean(df['Outlet Temp Error [C]']))

In [ ]:
data_path_low = norm(join(cwd, '..', '..', 'validation', 'MFRTRT', 'MFRTRT_Low_Flow_Loads.csv'))
df_raw_data_low = pd.read_csv(data_path_low, parse_dates=True, index_col='Date/Time')

In [ ]:
df_raw_data_low['time'] = pd.to_timedelta(df_raw_data_low.index)
df_raw_data_low['time'] = datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0) + (df_raw_data_low['time'] - df_raw_data_low['time'][0])
df_raw_data_low.set_index('time', inplace=True)
df_raw_data_low.index.rename('Date/Time', inplace=True)

In [ ]:
df_raw_data_low.to_csv(join(cwd, 'input_data_low.csv'))

In [ ]:
d_low = glhe.utilities.functions.load_json('MFRTRT_STS_input.json')
d_low['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, 'g.csv'))
d_low['temperature-profile'][0]['path'] = norm(join(cwd, 'input_data_low.csv'))
d_low['flow-profile'][0]['path'] = norm(join(cwd, 'input_data_low.csv'))
d_low['simulation']['output-path'] = cwd
d_low['simulation']['output-csv-name'] = 'out_low.csv'
d_low['simulation']['initial-temperature'] = 17.1
d_low['ground-temperature-model']['temperature'] = 17.1

d_low['simulation']['time-step'] = 15
d_low['simulation']['runtime'] = 3600 * 50
d_low['borehole-definitions'][0]['number-iterations'] = 2
d_low['borehole-definitions'][0]['fraction-grout'] = 0.25
d_low['borehole-definitions'][0]['segments'] = 1

In [ ]:
with open(join(cwd, 'in_low.json'), 'w') as f:
    f.write(json.dumps(d_low, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in_low.json').simulate()

In [ ]:
df_low = pd.read_csv('out_low.csv', index_col=0, parse_dates=True)

In [ ]:
start_time = '2019-01-03 00:00:00'
end_time = '2019-01-03 02:00:00'

ax = df_raw_data_low[['Exp. Inlet [C]', 'Exp. Outlet [C]']].loc[start_time:end_time].plot(style='x--')
df_low[['GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]']].loc[start_time:end_time].plot(ax=ax)
df_low[['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]']].loc[start_time:end_time].plot(ax=ax)
df_low[['GroundHeatExchangerSTS:GHE 1:Borehole Wall Temp. [C]']].loc[start_time:end_time].plot(ax=ax, style=':')

In [ ]:
df_low['Outlet Temp Error [C]'] = df_low['GroundHeatExchangerSTS:GHE 1:Outlet Temp. [C]'] - df_raw_data_low['Exp. Outlet [C]']

In [ ]:
'RMSE: {:0.4f} [C]'.format(np.mean(df_low['Outlet Temp Error [C]'] ** 2) ** 0.5)

In [ ]:
'MBE: {:0.4f} [C]'.format(np.mean(df_low['Outlet Temp Error [C]']))